# APRENDENDO SOBRE TTQ - TEXT TO QUERY  

Frameworks de agentes são fascinantes! Eles permitem a execução de uma série de tarefas que antes eram extremamente complicadas - ou até mesmo impossíveis.  
Trabalho criando consultas em bancos de dados desde 2018 e, quando os LLMs foram lançados, logo me perguntei:  

> Será que é possível pedir para uma LLM gerar uma consulta SQL e executá-la? 🤔  

Bem, é exatamente isso que vou testar neste notebook.  

## **Motivação**  

Uma das minhas principais atividades é digitalizar e automatizar processos de negócio. Dentro desse contexto, estabeleci um desafio:  

> Como criar um sistema que interprete uma solicitação em linguagem natural e gere uma consulta SQL sobre um processo?  

Acredito que esse objetivo é importante, pois, criar um sistema que transforma perguntas em **consultas SQL válidas** pode ser extremamente útil, uma vez que permitem:  

✅ **Melhorar a usabilidade** de sistemas, permitindo interações mais naturais com bases de dados. 

## Importando bibliotecas

In [1]:
from typing import List, Any, Dict
import re
import os
import sqlite3
import logging
import time
import json
from datetime import datetime
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_groq import ChatGroq
from dotenv import load_dotenv

## Carregando Variáveis de Ambiente

In [4]:
load_dotenv()

DB_PATH = "../.db/SQL_AGENT.db"

## Criando logger

In [2]:
logging.basicConfig(   
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[        
        logging.StreamHandler()
    ]
)

LOGGER = logging.getLogger(__name__)
LOGGER.setLevel(logging.INFO)

## Criando o banco de dados  

Antes de qualquer coisa, precisamos de um banco com uma tabela e algumas informações para pesquisar.  

Imagine uma empresa que gerencia processos como **Recrutamento**, **Seleção**, **Avaliação de Desempenho** e **Solicitação de Férias**, todos mapeados e digitalizados dentro da plataforma Lecom.  

Depois, foi criado um processo que extrai informações importantes sobre esses fluxos e as armazena na tabela `processos_andamento`. Essa será a base de dados utilizada pelo nosso sistema de agentes.  

Vale destacar que tudo isso foi criado de forma genérica, com a ajuda do ChatGPT. 🤖

In [5]:
LOGGER.info(f"Iniciando configuração do banco de dados em {DB_PATH}")

os.makedirs(os.path.dirname(DB_PATH), exist_ok=True)

try:
    conn = sqlite3.connect(DB_PATH)
    cursor = conn.cursor()
    LOGGER.info("Conexão com o banco de dados estabelecida com sucesso")
   
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table' AND name='processos_andamento'")
    if cursor.fetchone():       
        cursor.execute("DROP TABLE processos_andamento")
   
    LOGGER.info("Criando tabela 'processos_andamento'")
    cursor.execute("""
    CREATE TABLE processos_andamento (
        ID_Processo_Andamento INTEGER PRIMARY KEY AUTOINCREMENT,
        Codigo_Processo INTEGER NOT NULL,
        Codigo_Atividade INTEGER NOT NULL,
        Nome_Processo TEXT NOT NULL,
        Nome_Atividade TEXT NOT NULL,
        Nome_Cliente TEXT NOT NULL,
        Telefone_Cliente TEXT NOT NULL,
        Descricao_Processo TEXT NOT NULL,
        Data_Atividade DATE NOT NULL
    );
    """)
   
    dados_iniciais = [
        (1, 101, 'Recrutamento', 'Receber currículo', 'João Silva', '11987654321', 'Recebeu currículo e iniciou análise.', '2024-03-01'),
        (1, 102, 'Recrutamento', 'Entrevista inicial', 'João Silva', '11987654321', 'Entrevista marcada para avaliação inicial.', '2024-03-02'),
        (2, 201, 'Seleção', 'Teste técnico', 'Maria Oliveira', '11976543210', 'Teste técnico agendado.', '2024-03-03'),
        (2, 202, 'Seleção', 'Entrevista final', 'Carlos Pereira', '11965432109', 'Entrevista final marcada.', '2024-03-04'),
        (3, 301, 'Avaliação de Desempenho', 'Revisão do desempenho', 'Ana Souza', '11954321098', 'Coleta de feedbacks em andamento.', '2024-03-05'),
        (3, 302, 'Avaliação de Desempenho', 'Reunião de feedback', 'Carlos Pereira', '11965432109', 'Reunião agendada com gerente.', '2024-03-06'),
        (4, 401, 'Solicitação de Férias', 'Pedido formalizado', 'João Silva', '11987654321', 'Pedido de férias registrado.', '2024-03-07')
    ]
   
    LOGGER.info(f"Inserindo {len(dados_iniciais)} registros na tabela")
    cursor.executemany("""
        INSERT INTO processos_andamento
        (Codigo_Processo, Codigo_Atividade, Nome_Processo, Nome_Atividade, Nome_Cliente, Telefone_Cliente, Descricao_Processo, Data_Atividade)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?)
    """, dados_iniciais)
    
    # Verificar número de registros inseridos
    cursor.execute("SELECT COUNT(*) FROM processos_andamento")
    count = cursor.fetchone()[0]
    LOGGER.info(f"Total de registros na tabela: {count}")
    
    # Salvar e fechar a conexão
    conn.commit()
    conn.close()
    
    LOGGER.info(f"Banco de dados SQLite configurado com sucesso em {DB_PATH}")
    
except sqlite3.Error as e:
    LOGGER.error(f"Erro SQLite: {e}")
except Exception as e:
    LOGGER.error(f"Erro inesperado: {e}")

2025-03-24 22:15:01,664 - INFO - Iniciando configuração do banco de dados em ../.db/SQL_AGENT.db
2025-03-24 22:15:01,665 - INFO - Conexão com o banco de dados estabelecida com sucesso
2025-03-24 22:15:01,982 - INFO - Criando tabela 'processos_andamento'
2025-03-24 22:15:02,105 - INFO - Inserindo 7 registros na tabela
2025-03-24 22:15:02,106 - INFO - Total de registros na tabela: 7
2025-03-24 22:15:02,214 - INFO - Banco de dados SQLite configurado com sucesso em ../.db/SQL_AGENT.db


### Criando o gerenciador de banco de dados  

Ótimo! com a base de dados pronta, é hora de criar a classe responsável por manipulá-la.  

Essa classe é relativamente simples, mas essencial para projeto. Ela funciona como um intermediário entre os agentes e o banco de dados SQLite, permitindo que eles o manipulem.  

Os principais métodos são:  

- **`get_schema() -> str`**: Retorna o esquema das tabelas no banco, o que será útil para os agentes entenderem a estrutura dos dados.  
- **`execute_query(query: str) -> List[Any]`**: Executa uma query SQL e retorna os resultados. Se for uma consulta (`SELECT`), ela retorna os dados formatados como dicionários para facilitar a manipulação.  
- **`close()`**: Fecha a conexão com o banco de dados, garantindo que os recursos sejam liberados corretamente.  

In [6]:
class DatabaseManager:
    def __init__(self):
        """Inicializa o gerenciador de banco de dados com SQLite."""
        self.db_path = DB_PATH
        LOGGER.info(f"Inicializando DatabaseManager com banco de dados em {self.db_path}")
        try:
            self.connection = sqlite3.connect(self.db_path)
            self.connection.row_factory = sqlite3.Row  # Permite acessar os resultados por nome de coluna
            LOGGER.info("Conexão com o banco de dados estabelecida com sucesso\n")
        except sqlite3.Error as e:
            LOGGER.error(f"Erro ao conectar ao banco de dados: {str(e)}\n")
            raise Exception(f"Falha na conexão com o banco de dados: {str(e)}")

    def get_schema(self) -> str:
        """Recupera o esquema do banco de dados SQLite."""
        LOGGER.info("Obtendo esquema do banco de dados")
        try:
            cursor = self.connection.cursor()
            cursor.execute("SELECT name, sql FROM sqlite_master WHERE type='table';")
            schema_info = cursor.fetchall()
            
            tables_count = len(schema_info)
            LOGGER.debug(f"Encontradas {tables_count} tabelas no banco de dados")
            
            schema = "\n".join(f"Table: {row['name']}\n{row['sql']}" for row in schema_info if row['sql'])
            LOGGER.debug(f"Esquema obtido: {schema}\n")
            return schema
        except sqlite3.DatabaseError as e:
            error_msg = f"Erro ao obter o esquema do banco de dados: {str(e)}\n"
            LOGGER.error(error_msg)
            raise Exception(error_msg)

    def execute_query(self, query: str) -> List[Dict[str, Any]]:
        """Executa uma query SQL no banco SQLite e retorna os resultados."""
       
        LOGGER.info(f"Executando query: {query}")
        
        try:
            is_select = query.strip().lower().startswith("select")
            
            if not is_select:
                operacao = query.strip().split("/")[0]
                LOGGER.warning(f"Tentativa de execução de operação SQL '{operacao}' não permitida")
                raise Exception(f"Operação SQL '{operacao}' não permitida")
            
            cursor = self.connection.cursor()
            start_time = datetime.now()
            cursor.execute(query)
            
            results = [dict(row) for row in cursor.fetchall()]           
            
            execution_time = (datetime.now() - start_time).total_seconds()
            LOGGER.info(f"Tempo de execução: {execution_time:.3f} segundos\n")

            return results
        except sqlite3.DatabaseError as e:
            error_msg = f"Erro ao executar a consulta: {str(e)}\n"
            LOGGER.error(error_msg)            
            raise Exception(error_msg)

    def close(self):
        """Fecha a conexão com o banco de dados."""
        LOGGER.info("Fechando conexão com o banco de dados")
        try:
            self.connection.close()
            LOGGER.info("Conexão com o banco de dados fechada com sucesso\n")
        except sqlite3.Error as e:
            LOGGER.error(f"Erro ao fechar a conexão com o banco de dados: {str(e)}\n")

In [7]:
db_manager = DatabaseManager()

LOGGER.info(db_manager.get_schema())
LOGGER.info(db_manager.execute_query("SELECT * FROM processos_andamento")[0])

2025-03-24 22:15:17,784 - INFO - Inicializando DatabaseManager com banco de dados em ../.db/SQL_AGENT.db
2025-03-24 22:15:17,786 - INFO - Conexão com o banco de dados estabelecida com sucesso

2025-03-24 22:15:17,786 - INFO - Obtendo esquema do banco de dados
2025-03-24 22:15:17,787 - INFO - Table: sqlite_sequence
CREATE TABLE sqlite_sequence(name,seq)
Table: processos_andamento
CREATE TABLE processos_andamento (
        ID_Processo_Andamento INTEGER PRIMARY KEY AUTOINCREMENT,
        Codigo_Processo INTEGER NOT NULL,
        Codigo_Atividade INTEGER NOT NULL,
        Nome_Processo TEXT NOT NULL,
        Nome_Atividade TEXT NOT NULL,
        Nome_Cliente TEXT NOT NULL,
        Telefone_Cliente TEXT NOT NULL,
        Descricao_Processo TEXT NOT NULL,
        Data_Atividade DATE NOT NULL
    )
2025-03-24 22:15:17,787 - INFO - Executando query: SELECT * FROM processos_andamento
2025-03-24 22:15:17,788 - INFO - Tempo de execução: 0.000 segundos

2025-03-24 22:15:17,788 - INFO - {'ID_Proces

### Criando o gerenciador de LLM  

O que dá inteligência aos agentes é o LLM (Large Language Model), então nada mais justo do que criar um gerenciador específico para ele.  

Neste exemplo, será utilizado o modelo **`deepseek-r1-distill-llama-70b`** via **GROQ**, pois essa opção possui uma camada *free* bem generosa. Além disso, esse modelo tem a capacidade de **refletir antes de gerar uma resposta**, o que pode influenciar positivamente nos resultados. Bem, veremos se isso realmente faz diferença! 🤔  

O **LLMManager**, será responsável por interagir com o modelo e gerar respostas com base nos prompts fornecidos. Ele classe encapsula a comunicação com a API do modelo de LLM, permitindo uma interação simples e organizada.  

🔹 **`__init__()`**: Inicializa a conexão com a API da GROQ, configurando o modelo escolhido e os parâmetros principais:  
- `temperature=0.1`: Mantém as respostas mais determinísticas, reduzindo a criatividade excessiva.  
- `max_retries=2`: Define um limite de tentativas em caso de falha na requisição.  

🔹 **`invoke(prompt, **kwargs) -> str`**:  
- Recebe um `ChatPromptTemplate`, que contém o formato da mensagem.  
- Formata os dados necessários e envia a requisição para o LLM.  
- Retorna a resposta gerada pelo modelo. 

In [12]:
class LLMManager:
    def __init__(self):
        """Inicializa o gerenciador de LLM com o modelo Groq."""
        LOGGER.info("Inicializando LLMManager com o modelo deepseek-r1-distill-llama-70b")
        
        api_key = os.getenv("GROQ_API_KEY")
        if not api_key:
            LOGGER.error("GROQ_API_KEY não encontrada nas variáveis de ambiente\n")
            raise ValueError("GROQ_API_KEY não configurada. Configure a variável de ambiente GROQ_API_KEY.")
        
        try:
            self.llm = ChatGroq(
                model="deepseek-r1-distill-llama-70b",
                api_key=api_key,
                temperature=0.1,
                max_retries=2,
            )
            LOGGER.info("LLM inicializado com sucesso (modelo: deepseek-r1-distill-llama-70b, temperatura: 0.1)\n")
        except Exception as e:
            LOGGER.error(f"Erro ao inicializar o LLM: {str(e)}\n")
            raise Exception(f"Falha na inicialização do LLM: {str(e)}")

    def invoke(self, prompt: ChatPromptTemplate, **kwargs) -> str:
        """
        Invoca o LLM com o prompt fornecido e parâmetros adicionais.
        
        Args:
            prompt: O template de prompt do chat
            **kwargs: Variáveis para formatação do prompt
            
        Returns:
            str: A resposta do modelo
        """       
        
        try:           
            start_format_time = time.time()
            messages = prompt.format_messages(**kwargs)
            format_time = time.time() - start_format_time
            
            LOGGER.info("Enviando requisição ao modelo...")
            start_invoke_time = time.time()
            response = self.llm.invoke(messages)
            invoke_time = time.time() - start_invoke_time
           
            response_content = response.content
            LOGGER.debug(f"Resposta recebida em {invoke_time:.3f}s ({len(response_content)} caracteres)")
            LOGGER.info(f"Resposta do LLM: {response_content}")
            
            # Log de métricas
            total_time = format_time + invoke_time
            LOGGER.debug(f"Invocação completa. Tempo total: {total_time:.3f}s")
            
            return re.sub(r'<think>.*?</think>\s*', '', response_content, flags=re.DOTALL)
            
        except Exception as e:
            LOGGER.error(f"Erro ao invocar o LLM: {str(e)}")
            raise Exception(f"Falha na invocação do LLM: {str(e)}")

In [13]:
llm_manager = LLMManager()

template = ChatPromptTemplate([
    ("system", "Seu nome é Brian, você está sempre feliz e alegre, sempre respondendo em PT-BR."),
    ("human", "Olá, meu nome é Rodrigo, e o seu?"),   
])

llm_manager.invoke(template)

2025-03-24 22:18:55,235 - INFO - Inicializando LLMManager com o modelo deepseek-r1-distill-llama-70b
2025-03-24 22:18:55,268 - INFO - LLM inicializado com sucesso (modelo: deepseek-r1-distill-llama-70b, temperatura: 0.1)

2025-03-24 22:18:55,270 - INFO - Enviando requisição ao modelo...
2025-03-24 22:18:56,329 - INFO - Resposta do LLM: <think>
Okay, so Rodrigo just introduced himself and asked my name. I need to respond in a friendly and happy manner. Since I'm supposed to always be cheerful, I should keep the tone upbeat. I'll start by greeting him back and sharing my name, Brian. Then, I'll express excitement about meeting him. Maybe add an emoji to keep it lively. Let me put that together.
</think>

Olá Rodrigo! Eu sou o Brian! Muito prazer em te conhecer! 😊


'Olá Rodrigo! Eu sou o Brian! Muito prazer em te conhecer! 😊'

### Analisando a Pergunta do Usuário  

Ao lidar com um problema envolvendo **banco de dados**, a primeira tarefa é identificar **quais tabelas e colunas** fazem parte da solução. Neste caso, existe apenas **uma tabela**, pois toda a lógica para populá-la será resolvida sem a necessidade de um sistema de agentes para isso.  

Com essa base estabelecida, é possível testar se o **Agente** criado, utilizando o **LLM**, consegue **abstrair a lógica necessária** para interpretar uma pergunta e retornar as colunas relevantes.  

### 🚀 Técnicas Essenciais de Prompt Engineering  

Ao usar **técnicas de engenharia de prompt (Prompt Engineering)**, é possível melhorar a capacidade do LLM de entender e gerar consultas corretas.  

Para criar um agente eficiente, deve-se utilizar algumas estratégias fundamentais:  

🔹 **Definição de Persona**: Faz com que o modelo assuma o papel de um **analista de dados**, influenciando seu estilo de resposta.  
🔹 **Instrução Clara e Específica**: Detalha exatamente o que é esperado do modelo, reduzindo ambiguidades.  
🔹 **Formato de Saída Especificado**: Garante que o modelo retorne um **JSON estruturado**, facilitando o processamento da resposta.  
🔹 **Restrições e Regras Detalhadas**: Define os limites para que o modelo foque nas colunas **relevantes**, ignorando campos irrelevantes.  
🔹 **Injeção de Contexto**: É passado o **esquema do banco de dados** para que o modelo compreenda melhor a estrutura disponível.  
🔹 **Uso de Delimitadores**: Organiza a entrada do modelo, separando **esquema do banco** e **pergunta do usuário**, melhorando a compreensão.

💡 Essa abordagem permite que o modelo compreenda a **estrutura do banco** e selecione apenas as colunas relevantes com mais eficiência! 🚀

In [14]:
def analisar_pergunta(pergunta: str) -> Dict[str, Any]:
    """
    Analisa a pergunta do usuário e identifica as tabelas e colunas relevantes.
    
    Args:
        pergunta (str): Pergunta do usuário a ser analisada
        
    Returns:
        Dict[str, Any]: Dicionário contendo a análise da pergunta
    """    

    LOGGER.info(f"Iniciando análise da pergunta: '{pergunta}'")
    
    try:        
        start_schema_time = time.time()
        esquema = db_manager.get_schema()
        
        LOGGER.debug(f"Preparando prompt para o LLM")
        prompt = ChatPromptTemplate.from_messages([
            ("system", '''Você é um analista de dados que pode ajudar a resumir tabelas SQL e interpretar perguntas de usuários sobre um banco de dados.  
Dada a pergunta e o esquema do banco de dados, identifique as tabelas e colunas relevantes.  
Se a pergunta não for relevante para o banco de dados ou se não houver informações suficientes para respondê-la, defina "is_relevante" como false.
Sua resposta deve estar no seguinte formato JSON:
{{
    "is_relevante": boolean,
    "tabelas_relevantes": [
        {{
            "nome_tabela": string,
            "colunas": [string],
            "colunas_substantivo": [string]
        }}
    ]
}}
O campo "colunas_substantivo" deve conter apenas as colunas que são relevantes para a pergunta e que contêm substantivos ou nomes.  
Por exemplo, a coluna "Nome do Artista" contém substantivos relevantes para a pergunta "Quais são os artistas mais vendidos?",  
mas a coluna "ID do Artista" não é relevante, pois não contém um substantivo. Não inclua colunas que contenham números.
'''),
            ("human", "===Esquema do banco de dados:\n{esquema}\n\n===Pergunta do usuário:\n{pergunta}\n\nIdentifique as tabelas e colunas relevantes:")
        ])        
       
        analisador_json = JsonOutputParser()
         
        resposta = llm_manager.invoke(prompt, esquema=esquema, pergunta=pergunta)          

        LOGGER.debug(f"Analisando resposta JSON")      
        try:
            resposta_analisada = analisador_json.parse(resposta) 
            
            is_relevante = resposta_analisada.get("is_relevante", False)
            num_tabelas = len(resposta_analisada.get("tabelas_relevantes", []))
            LOGGER.info(f"Análise concluída. Relevante: {is_relevante}, Tabelas identificadas: {num_tabelas}")
            
            if num_tabelas > 0:
                tabelas_nomes = [tabela.get("nome_tabela") for tabela in resposta_analisada.get("tabelas_relevantes", [])]
                LOGGER.debug(f"Tabelas relevantes: {', '.join(tabelas_nomes)}")
            
        except Exception as e:
            LOGGER.error(f"Erro ao analisar JSON da resposta: {str(e)}")
            LOGGER.error(f"Resposta que causou o erro: {resposta}\n")
            # Se houver erro no parsing, retornamos uma resposta de fallback
            resposta_analisada = {"is_relevante": False, "tabelas_relevantes": []}
        
        LOGGER.info(f"Resultado Final da Análise: {str(resposta_analisada)}")
        
        # Tempo total da operação
        total_time = time.time() - start_schema_time
        LOGGER.info(f"Análise completa em {total_time:.3f}s")
        
        # Retornar o resultado
        return {"pergunta_analisada": resposta_analisada}
        
    except Exception as e:
        LOGGER.error(f"Erro durante análise da pergunta: {str(e)}\n")
        
        return {"pergunta_analisada": {"is_relevante": False, "tabelas_relevantes": []}}

In [15]:
pergunta = "Do que se trada o processo da Ana Souza?"
pergunta_analisada = analisar_pergunta(pergunta)['pergunta_analisada']

2025-03-24 22:23:44,259 - INFO - Iniciando análise da pergunta: 'Do que se trada o processo da Ana Souza?'
2025-03-24 22:23:44,260 - INFO - Obtendo esquema do banco de dados
2025-03-24 22:23:44,261 - INFO - Enviando requisição ao modelo...
2025-03-24 22:23:47,959 - INFO - Resposta do LLM: <think>
Ok, vamos analisar a pergunta do usuário: "Do que se trada o processo da Ana Souza?" Primeiro, preciso entender o que o usuário está procurando. Parece que ele quer saber qual é o processo relacionado a uma pessoa específica, Ana Souza.

Agora, vou verificar o esquema do banco de dados fornecido. Existem duas tabelas: sqlite_sequence e processos_andamento. A tabela sqlite_sequence parece ser do sistema e não contém informações relevantes para a pergunta, então posso ignorá-la.

Focando na tabela processos_andamento, vejo que ela tem várias colunas: ID_Processo_Andamento, Codigo_Processo, Codigo_Atividade, Nome_Processo, Nome_Atividade, Nome_Cliente, Telefone_Cliente, Descricao_Processo e Data_

### Encontrando Substantivos Únicos em Tabelas e Colunas Relevantes

Ao utilizar um LLM para processar consultas, é essencial validar a resposta gerada. Essa etapa do processo tem como objetivo higienizar o retorno, garantindo que cada coluna seja referenciada apenas uma vez.

In [25]:
def obter_substantivos_unicos(pergunta_analisada: dict) -> dict:
    """Identifica substantivos únicos nas tabelas e colunas relevantes."""
    
    if not pergunta_analisada['is_relevante']:
        LOGGER.info("A pergunta não é relevante. Retornando lista vazia.\n")
        return {"substantivos_unicos": []}

    substantivos_unicos = set()
    LOGGER.info("Iniciando a busca por substantivos únicos.")

    for info_tabela in pergunta_analisada['tabelas_relevantes']:
        nome_tabela = info_tabela['nome_tabela']
        colunas_substantivos = info_tabela['colunas_substantivo']

        if not colunas_substantivos:
            LOGGER.debug(f"A tabela '{nome_tabela}' não possui colunas relevantes.")
            continue

        nomes_colunas = ', '.join(f"`{col}`" for col in colunas_substantivos)
        consulta = f"SELECT DISTINCT {nomes_colunas} FROM `{nome_tabela}`"
        LOGGER.debug(f"Executando consulta SQL na tabela '{nome_tabela}': {consulta}")

        try:
            resultados = db_manager.execute_query(consulta)
            LOGGER.debug(f"Consulta retornou {len(resultados)} registros.")

            for linha in resultados:
                valores = [str(valor) for valor in linha if valor]
                substantivos_unicos.update(valores)
                LOGGER.debug(f"Valores extraídos: {valores}")

        except Exception as e:
            LOGGER.error(f"Erro ao executar consulta na tabela '{nome_tabela}': {e}\n")

    LOGGER.info(f"Processo concluído. {len(substantivos_unicos)} substantivos únicos encontrados.")
    LOGGER.info(f"Substantivos Únicos: {str(substantivos_unicos)}\n")
    return {"substantivos_unicos": list(substantivos_unicos)}


In [26]:
substantivos_unicos = obter_substantivos_unicos(pergunta_analisada)['substantivos_unicos']

2025-03-24 22:33:52,146 - INFO - Iniciando a busca por substantivos únicos.
2025-03-24 22:33:52,146 - INFO - Executando query: SELECT DISTINCT `Nome_Cliente`, `Nome_Processo`, `Descricao_Processo` FROM `processos_andamento`
2025-03-24 22:33:52,147 - INFO - Tempo de execução: 0.001 segundos

2025-03-24 22:33:52,148 - INFO - Processo concluído. 3 substantivos únicos encontrados.
2025-03-24 22:33:52,148 - INFO - Substantivos Únicos: {'Nome_Processo', 'Nome_Cliente', 'Descricao_Processo'}



### ✨ Gerando uma consulta SQL com base na pergunta analisada e nos substantivos únicos  

Sabe aquele momento em que o filme chega ao seu ápice? 🎬 Pois é, estamos exatamente aí! Já temos a pergunta do usuário, uma análise das colunas relevantes e as colunas devidamente higienizadas. Ou seja, temos insumos suficientes para criar um agente que irá gerar a consulta SQL. *Maravilhindo!* 🚀  

Aproveitando o embalo, vamos falar sobre mais uma técnica de prompt:  

- **Few-shot Prompting (Exemplos)**: Essa técnica consiste em fornecer ao modelo alguns exemplos de entrada e suas respectivas saídas antes da pergunta principal. Isso ajuda o modelo a entender o formato esperado da resposta e a aprender o padrão da tarefa, melhorando a qualidade da geração. 🎯  

In [29]:
def gerar_sql(pergunta: str, pergunta_analisada: dict, substantivos_unicos: list) -> dict:
    """Gera uma consulta SQL com base na pergunta analisada e nos substantivos únicos."""
    
    LOGGER.info(f"Iniciando geração da consulta SQL para a pergunta: '{pergunta}'")
    start_time = time.time()

    if not pergunta_analisada['is_relevante']:
        LOGGER.warning("Pergunta não é relevante.")
        return {"consulta_sql": "NAO_RELEVANTE", "is_relevante": False}
    
    try:
        esquema = db_manager.get_schema()
        
        LOGGER.debug("Preparando prompt para o LLM")
        prompt = ChatPromptTemplate.from_messages([
            ("system", '''
    Você é um assistente de IA que gera consultas SQL com base na pergunta do usuário, no esquema do banco de dados e nos substantivos únicos encontrados nas tabelas relevantes. Gere uma consulta SQL válida para responder à pergunta do usuário.

    Se não houver informações suficientes para escrever uma consulta SQL, responda com "INFORMACAO_INSUFICIENTE".

    Aqui estão alguns exemplos:

    1. Qual é o produto mais vendido?
    Resposta: SELECT product_name, SUM(quantity) as total_quantity FROM sales WHERE product_name IS NOT NULL AND quantity IS NOT NULL AND product_name != "" AND quantity != "" AND product_name != "N/A" AND quantity != "N/A" GROUP BY product_name ORDER BY total_quantity DESC LIMIT 1

    2. Qual é a receita total para cada produto?
    Resposta: SELECT \`product name\`, SUM(quantity * price) as total_revenue FROM sales WHERE \`product name\` IS NOT NULL AND quantity IS NOT NULL AND price IS NOT NULL AND \`product name\` != "" AND quantity != "" AND price != "" AND \`product name\` != "N/A" AND quantity != "N/A" AND price != "N/A" GROUP BY \`product name\`  ORDER BY total_revenue DESC

    3. Qual é a participação de mercado de cada produto?
    Resposta: SELECT \`product name\`, SUM(quantity) * 100.0 / (SELECT SUM(quantity) FROM sales) as market_share FROM sales WHERE \`product name\` IS NOT NULL AND quantity IS NOT NULL AND \`product name\` != "" AND quantity != "" AND \`product name\` != "N/A" AND quantity != "N/A" GROUP BY \`product name\`  ORDER BY market_share DESC
   
    Apenas forneça a string da consulta SQL. Não a formate. Certifique-se de usar a grafia correta dos substantivos conforme fornecido na lista de substantivos únicos. Todos os nomes de tabelas e colunas devem estar entre crases.
    '''),
            ("human", '''===Esquema do banco de dados:
    {esquema}

    ===Pergunta do usuário:
    {pergunta}

    ===Tabelas e colunas relevantes:
    {pergunta_analisada}

    ===Substantivos únicos nas tabelas relevantes:
    {substantivos_unicos}

    Gere a string da consulta SQL'''),
        ])
       
        consulta = llm_manager.invoke(
            prompt, 
            esquema=esquema, 
            pergunta=pergunta, 
            pergunta_analisada=pergunta_analisada, 
            substantivos_unicos=substantivos_unicos
        )

        if consulta.strip() == "INFORMACAO_INSUFICIENTE":
            return {"consulta_sql": "NAO_RELEVANTE"}
        
        total_time = time.time() - start_time
        LOGGER.info(f"Consulta SQL gerada com sucesso em {total_time:.3f}s")
        LOGGER.info(f"Consulta SQL: {consulta}\n")
        
        return {"consulta_sql": consulta}
        
    except Exception as e:
        LOGGER.error(f"Erro durante a geração da consulta SQL: {str(e)}\n")
        return {"consulta_sql": "ERRO_EXECUCAO", "is_relevante": False}



In [30]:
consulta_sql = gerar_sql(pergunta, pergunta_analisada, substantivos_unicos)['consulta_sql']

2025-03-24 22:56:57,586 - INFO - Iniciando geração da consulta SQL para a pergunta: 'Do que se trada o processo da Ana Souza?'
2025-03-24 22:56:57,587 - INFO - Obtendo esquema do banco de dados
2025-03-24 22:56:57,851 - INFO - Enviando requisição ao modelo...
2025-03-24 22:56:59,973 - INFO - Resposta do LLM: <think>
Okay, I need to help the user by generating an SQL query based on their question. The question is "Do que se trada o processo da Ana Souza?" which translates to "What is Ana Souza's process about?" 

First, I look at the database schema provided. There's a table called processos_andamento with several columns. The relevant columns for this question are Nome_Processo, Nome_Cliente, and Descricao_Processo. 

The user wants to know about Ana Souza's process, so I need to filter the records where Nome_Cliente is 'Ana Souza'. That means I'll use a WHERE clause to select rows where Nome_Cliente equals 'Ana Souza'.

Next, I should retrieve the relevant information. Since the quest

## ✅ Validando e Corrigindo a Consulta SQL Gerada  

Se sua memória for boa, você deve se lembrar que validar e corrigir a resposta de um LLM é uma prática recomendada. Essa abordagem tem sido amplamente adotada pela comunidade, garantindo maior precisão e confiabilidade nas respostas.  

Portanto, antes de utilizarmos a consulta SQL gerada, vamos validar se o outro agente trabalhou como esperado.  

As técnicas utilizadas aqui não são nenhuma novidade, mas são essenciais para evitar erros e garantir um resultado mais confiável. ⚡  


Agora, com esse processo de validação e correção, garantimos que a consulta SQL gerada esteja correta antes de ser executada. 🔍💡

In [ ]:
def validar_e_corrigir_sql(consulta_sql) -> dict:
    """Valida e corrige a consulta SQL gerada."""
           
    LOGGER.info(f"Iniciando validação da consulta SQL: '{consulta_sql}'")
    start_time = time.time()
    
    if consulta_sql == "NAO_RELEVANTE":
        LOGGER.warning("A consulta não é relevante.")
        return {"consulta_sql": "NAO_RELEVANTE", "consulta_valida": False}
    
    try:
        
        esquema = db_manager.get_schema()

        prompt = ChatPromptTemplate.from_messages([
            ("system", '''
    Você é um assistente de IA que valida e corrige consultas SQL. Sua tarefa é:
    1. Verificar se a consulta SQL é válida.
    2. Garantir que todos os nomes de tabelas e colunas estejam corretamente escritos e existam no esquema do banco de dados. Todos os nomes de tabelas e colunas devem estar entre crases.
    3. Se houver problemas, corrija-os e forneça a consulta SQL corrigida.
    4. Se não houver problemas, retorne a consulta original.

   Responda no formato JSON com a seguinte estrutura. Responda apenas com o JSON:
    {{
        "valido": booleano,
        "problemas": string ou null,
        "consulta_corrigida": string
    }}

    Por exemplo:
    1. 
    {{
        "valido": true,
        "problemas": null,
        "consulta_corrigida": "None"
    }}
                
    2. 
    {{
        "valido": false,
        "problemas": "A coluna USERS não existe",
        "consulta_corrigida": "SELECT * FROM users WHERE age > 25"
    }}

    3. 
    {{
        "valido": false,
        "problemas": "Os nomes de colunas e tabelas devem estar entre crases se contiverem espaços",
        "consulta_corrigida": "SELECT * FROM \`gross income\` WHERE \`age\` > 25"
    }}
    '''),
            ("human", '''===Esquema do banco de dados:
    {esquema}

    ===Consulta SQL gerada:
    {consulta_sql}
                
    '''),
        ])

        analisador_saida = JsonOutputParser()
        
        resposta = llm_manager.invoke(prompt, esquema=esquema, consulta_sql=consulta_sql)
        resultado = analisador_saida.parse(resposta)

        if resultado["valido"] and resultado["problemas"] is None:
            total_time = time.time() - start_time
            LOGGER.info(f"Consulta SQL validade em {total_time:.3f}s")
            LOGGER.info(f"Consulta SQL: {resultado['consulta_corrigida']}\n")
            
            return {"consulta_corrigida": consulta_sql, "consulta_valida": True, "consulta_problemas": None}
        
        total_time = time.time() - start_time
        LOGGER.info(f"Consulta SQL validade em {total_time:.3f}s")
        LOGGER.info(f"Consulta SQL: {resultado['consulta_corrigida']}\n")
        
        
        return {
            "consulta_corrigida": resultado["consulta_corrigida"],
            "consulta_valida": resultado["valido"],
            "consulta_problemas": resultado["problemas"]
        }
    except Exception as e:
        LOGGER.error(f"Erro ao validar a consulta SQL: {e}\n", exc_info=True)
        return {"consulta_sql": "ERRO_EXECUCAO", "consulta_valida": False, "consulta_problemas": str(e)}


In [55]:
consulta_corrigida = validar_e_corrigir_sql(consulta_sql)
LOGGER.info(consulta_corrigida)

2025-03-24 23:37:08,177 - INFO - Iniciando validação da consulta SQL: 'SELECT \`Nome_Processo\`, \`Descricao_Processo\` FROM \`processos_andamento\` WHERE \`Nome_Cliente\` = 'Ana Souza''
2025-03-24 23:37:08,177 - INFO - Obtendo esquema do banco de dados
2025-03-24 23:37:08,178 - INFO - Enviando requisição ao modelo...
2025-03-24 23:37:13,090 - INFO - Resposta do LLM: <think>
Okay, I need to validate and correct the given SQL query. Let me start by understanding the task. The user provided a SQL query and the database schema, and I have to check if the query is valid, correct any issues, and return the result in a specific JSON format.

First, I'll look at the database schema. There's a table called processos_andamento with several columns: ID_Processo_Andamento, Codigo_Processo, Codigo_Atividade, Nome_Processo, Nome_Atividade, Nome_Cliente, Telefone_Cliente, Descricao_Processo, and Data_Atividade. All these columns are correctly defined, so I can refer to them.

Now, the SQL query prov

### 🚀 Executando a consulta SQL gerada 🔍
Agora saberemos se o objetivo foi alcançado ✅. Nada de LLM, somente a boa e velha QUERY ⚡💻.

In [51]:
def executar_sql(consulta: str) -> dict:
    """Executa a consulta SQL e retorna os resultados."""
    
    LOGGER.info(f"Iniciando a execução da consulta SQL: '{consulta_sql}'")
    start_time = time.time()
    
    if consulta == "NAO_RELEVANTE":
        LOGGER.warning("A consulta não é relevante.")
        return {"resultados": "NAO_RELEVANTE"}

    try:
        resultados = db_manager.execute_query(consulta)
        
        total_time = time.time() - start_time
        LOGGER.info(f"Consulta SQL exedcutada em {total_time:.3f}s")
        LOGGER.info(f"Resultado: {str(resultados)}\n")
        
        return {"resultados": resultados}
    except Exception as e:
        LOGGER.error(f"Erro ao executar a consulta SQL: {e}\n", exc_info=True)
        return {"erro": str(e)}

In [52]:
resultados = executar_sql(consulta_corrigida['consulta_corrigida']) 
colunas = resultados["resultados"][0].keys()

print(" | ".join(colunas))
print("-" * (len(" | ".join(colunas)) + 5))

for linha in resultados["resultados"]:
    print(" | ".join(str(valor) for valor in linha.values()))

2025-03-24 23:36:00,229 - INFO - Iniciando a execução da consulta SQL: 'SELECT \`Nome_Processo\`, \`Descricao_Processo\` FROM \`processos_andamento\` WHERE \`Nome_Cliente\` = 'Ana Souza''
2025-03-24 23:36:00,230 - INFO - Executando query: SELECT \`Nome_Processo\`, \`Descricao_Processo\` FROM \`processos_andamento\` WHERE \`Nome_Cliente\` = 'Ana Souza'
2025-03-24 23:36:00,231 - ERROR - Erro ao executar a consulta: unrecognized token: "\"

2025-03-24 23:36:00,231 - ERROR - Erro ao executar a consulta SQL: Erro ao executar a consulta: unrecognized token: "\"

Traceback (most recent call last):
  File "/tmp/ipykernel_2029/2709870297.py", line 48, in execute_query
    cursor.execute(query)
sqlite3.OperationalError: unrecognized token: "\"

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/tmp/ipykernel_2029/603843889.py", line 12, in executar_sql
    resultados = db_manager.execute_query(consulta)
                 ^^^^^^^^^^^^^^

KeyError: 'resultados'

## Conclusão

Aparentemente os resultados foram bons, o processo criado abstrai bem a lógica de criação de um SQL, o LLM conseguiu realizar muito bem sua atividade, e, ao menos para mim, a resposta foi o esperado.

Claro que quanto mais os Agentes forem refinados, mais a resposta ficará melhor.

## Próximos Passos

Um ponto final só é o começo de uma nova frase, seguindo esse analogia, quero continuar evoluindo esse projeto. Algumas funcionalidades que irei implementar:

- Criar um Agente que cria uma resposta para o usuário.
- Criar um Grafo com lang-graf.
- Criar um Chat utilizando Chainlit.
- Quem sabe: Criar um Chatbot no Whatssapp com esse esquema.


## Minhas Limitações

Ainda tenho muitas duvidas quanto o processo de criar consultas a partir de texto natural, o TTQ(text to Query) ainda me parece um pouco complicado, mas isso vai melhorar ao longo do tempo.

- Como lidar com várias requisições vindas pelos Whatsapp
- Como manter o contexto em uma conversa, o usuário pode fazer uma pergunta e depois outra.

## Como Imagino Parte da Implementação

Já parti do principio que existe uma tabela com toda as informações necessárias. Por obvio ela deverá ser criada e as informações carregadas nela de alguma forma. Parte da lógica que usária é a seguinte:

- Criar a tabela que irá receber os processo.
- Cada processo criado dentro da ferramenta é responsável por registar seu progresso nessa tabela, pode ser com uma integração passagem de etapa.
  - Ao abrir o processo devesse registar o código dele nessa tabela
  - Ao passa cada atividade, ou ao menos as mais relevantes para consulta, buscar pela referencia do processo na tabela e atualizar as informações.
- Como algumas informações podem ser somente daquele processo, e não caber em um coluna genêrica, pode-se criar uma coluna que contenha um JSON com algumas informações especificas, essas informações podem servir de insumos para o Agente que irá criar a resposta.

Para lidar com várias requisições pode ser que de para usar uma fila, por exemplo Kafka.